# Simulating decays in pythia using chromo's fast vectorized interface

This example demonstrates how to use the `DecayHandler` class from chromo to simulate decays using pythia 8 decay routines. 

In [3]:
from chromo.kinematics import FixedTarget, KinEnergy
from chromo.decay_handler import Pythia8DecayHandler
from chromo.common import EventData
import numpy as np
import matplotlib.pyplot as plt
from particle import Particle
import boost_histogram as bh
from chromo.util import EventFrame


In [4]:
# These are all hadrons up to bottom, plus leptons and photons
stable_list = Particle.findall(
    lambda p: not (
        abs(p.pdgid) < 11
        or abs(p.pdgid) in [21, 22]
        or (22 < abs(p.pdgid) < 111)
        or abs(p.pdgid) > 5000
        or p.pdgid.has_bottom
        or (p.mass is None)
        or (p.pdgid.is_lepton and abs(p.pdgid) not in [15, 13])
    ),
    particle=True,
)

In [6]:
counts = 0
for p in stable_list:
    print(p, p.mass, p.pdgid)
    counts += 1
    if counts > 20:
        break

mu- 105.6583755 <PDGID: 13>
tau- 1776.93 <PDGID: 15>
pi0 134.9768 <PDGID: 111>
rho(770)0 775.26 <PDGID: 113>
a(2)(1320)0 1318.2 <PDGID: 115>
rho(3)(1690)0 1688.8 <PDGID: 117>
a(4)(1970)0 1967.0 <PDGID: 119>
K(L)0 497.611 <PDGID: 130>
pi+ 139.57039 <PDGID: 211>
rho(770)+ 775.11 <PDGID: 213>
a(2)(1320)+ 1318.2 <PDGID: 215>
rho(3)(1690)+ 1688.8 <PDGID: 217>
a(4)(1970)+ 1967.0 <PDGID: 219>
eta 547.862 <PDGID: 221>
omega(782) 782.66 <PDGID: 223>
f(2)(1270) 1275.4 <PDGID: 225>
omega(3)(1670) 1667.0 <PDGID: 227>
f(4)(2050) 2018.0 <PDGID: 229>
K(S)0 497.611 <PDGID: 310>
K0 497.611 <PDGID: 311>
K*(892)0 895.55 <PDGID: 313>


In [7]:
len(stable_list)

105

In [8]:
py8decays = Pythia8DecayHandler(stable_pids=[p.pdgid for p in stable_list])

In [47]:
def put_on_stack(projectile, n_copies=1):
    # Customize arguments if want to pass custom energy or momenta
    # Pass en, pz, etc. further down below
    m = projectile.mass * 1e-3
    # Silence warning
    with np.errstate(invalid='ignore'):
        kin = FixedTarget(KinEnergy(0.0), projectile.pdgid, 2212)
    # Initialize an EventData object with empty arrays
    event = EventData(
        generator=("", ""),  # Empty generator name and version
        kin=kin,  # kinematics
        nevent=0,  # Event number
        impact_parameter=0.0,  # Impact parameter in mm
        n_wounded=(0, 0),  # Number of wounded nucleons on sides A and B
        production_cross_section=0.0,  # Production cross section in mb
        pid=np.zeros(n_copies, dtype=int),  # Empty PDG ID array
        status=np.zeros(n_copies, dtype=int),  # Empty status array
        charge=np.zeros(n_copies, dtype=float),  # Empty charge array
        px=np.zeros(n_copies, dtype=float),  # Empty px momentum array
        py=np.zeros(n_copies, dtype=float),  # Empty py momentum array
        pz=np.zeros(n_copies, dtype=float),  # Empty pz momentum array
        en=np.zeros(n_copies, dtype=float),  # Empty energy array
        m=np.zeros(n_copies, dtype=float),  # Empty mass array
        vx=np.zeros(n_copies, dtype=float),  # Empty vx position array
        vy=np.zeros(n_copies, dtype=float),  # Empty vy position array
        vz=np.zeros(n_copies, dtype=float),  # Empty vz position array
        vt=np.zeros(n_copies, dtype=float),  # Empty vt time array
        mothers=None,  # No mother information
        daughters=None,  # No daughter information
    )
    event.en[:] = m
    event.m[:] = m
    event.pid[:] = projectile.pdgid
    event.status[:] = 1

    return event

In [48]:
# Choose projectile
pid = 321  # K+
projectile = Particle.from_pdgid(pid)

In [49]:
print(f"Particle {projectile} is in stable_list {projectile in stable_list}")
# Make unstable so it can decay in pythia
py8decays.pythia.particleData.mayDecay(pid, True)

Particle K+ is in stable_list True


In [50]:
# Optional: customize options using standard pythia string interface
# py8decays.pythia.readString("321:onMode = on")  # activate all channels
# py8decays.pythia.readString(
#     "321:offIfMatch = -13 14"
# ) # Disable 2-body decays

In [51]:
# Make a new event and put n_copies projectiles on the stack (if you want to let many of the particles decay)
event = put_on_stack(projectile, n_copies=1)  # Use a single particle to decay here
# Run pythia decays
py8decays(event)

In [53]:
# Look only at final state particles
ev = event.final_state()
# Access the event in standard chromo way.
# The decay products are always appended to the end of the original arrays with status 1 (no
# need to care about this if using final_state() as shown here.)
for i, p in enumerate(range(len(ev.pid))):
    print(i, ev.pid[i], ev.elab[i], ev.pz[i])


0 211 0.14636128053809327 -0.04276702215882962
1 211 0.18480564612656492 0.11068673200545695
2 -211 0.16251007333534181 -0.06791970984662735
